## 5-fold cross-validation

In [1]:
# import prepare_data

# from Feature import Feature
# from Decoder import Decoder
# import operator

# mode = 'train_avg'
# iter = 10
# beam_size = 16


### original definition of function "train_avg"

In [2]:
# def train_avg(iterations, train_file, beam_size):
#     data = prepare_data.read_file(train_file)
#     feature = Feature()
#     decoder = Decoder(beam_size, feature.get_score)
#     n = 0
#     for t in range(iterations):
#         count = 0
#         data_size = len(data)

#         for line in data:
#             n += 1
#             y = line.split()
#             z = decoder.beamSearch(line)
#             if z != y:
#                 feature.update_avgWeight(y, z, n, t, data_size)

#             train_seg = ' '.join(z)

#             count += 1
#             if count % 1000 == 0:
#                 print("iter %d , finish %.2f%%" %
#                       (t, (count / data_size) * 100))

#         model_file = open("cv-model_result/model-" +
#                             str(t) + "_beam-size-" + str(beam_size) + '.pkl', 'wb')
#         feature.save_model(model_file)
#         model_file.close()
#         print("segment with model-%d finish" % t)
#         print("iteration %d finish" % t)

#     feature.last_update(iterations, data_size)
#     feature.cal_avg_weight(iterations, data_size)
#     avg_model = open(
#         "cv-model_result/avg-model_beam-size-" + str(beam_size) + '.pkl', 'wb')
#     feature.save_model(avg_model)
#     avg_model.close()
#     print("segment with avg-model finish")


In [3]:
# train_file = 'data/filter_train.txt'
# test_file = 'data/filter_test.txt'

# train_avg(iter, train_file, beam_size)

### my training process

In [4]:
# import sklearn
# import numpy as np
# import prepare_data

# from Feature import Feature
# from Decoder import Decoder
# from sklearn.model_selection import KFold

# kf = KFold(n_splits=5, shuffle=True)

# train_file = 'data/fusion_data.txt'
# raw_data = prepare_data.read_file(train_file)

# for train_id, test_id in kf.split(raw_data):
#     print("TRAIN:", train_id, "TEST:", test_id)
#     train_data, test_data = np.array(raw_data)[train_id], np.array(raw_data)[test_id]
#     print(len(raw_data), len(train_data), len(test_data))
#     break

In [5]:
# test_file = 'data/filter_test.txt'
# raw_data = prepare_data.read_file(test_file)

# sent = raw_data[0]
# split_sent = [char for char in sent if(char != ' ') and (char != '\n')]
# split_sent

In [6]:
def train_avg(iterations, train_data, beam_size, cv_epoch_num):
    import logging
    # data = prepare_data.read_file(train_file)
    data = train_data
    feature = Feature()
    decoder = Decoder(beam_size, feature.get_score)
    n = 0
    for t in range(iterations):
        count = 0
        data_size = len(data)

        for line in data:
            n += 1
            y = line.split()
            z = decoder.beamSearch(line)
            if z != y:
                feature.update_avgWeight(y, z, n, t, data_size)

            train_seg = ' '.join(z)

            count += 1
            if count % 1000 == 0:
                print("iter %d , finish %.2f%%" % \
                        (t, (count / data_size) * 100))
                logging.info("iter %d , finish %.2f%%" % \
                        (t, (count / data_size) * 100))

        model_file = open('cv-' + str(cv_epoch_num) + "-model_result/model-" + \
                            str(t) + "_beam-size-" + str(beam_size) + '.pkl', 'wb')
        feature.save_model(model_file)
        model_file.close()
        print("segment with model-%d finish" % t)
        logging.info("segment with model-%d finish" % t)
        print("iteration %d finish" % t)
        logging.info("iteration %d finish" % t)

    feature.last_update(iterations, data_size)
    feature.cal_avg_weight(iterations, data_size)
    avg_model = open(
        'cv-' + str(cv_epoch_num) + "-model_result/avg-model_beam-size-" + str(beam_size) + '.pkl', 'wb')
    feature.save_model(avg_model)
    avg_model.close()
    print("segment with avg-model finish")
    logging.info("segment with avg-model finish")

In [7]:
def test_avg(iterations, test_data, beam_size, cv_epoch_num):
    import logging
    # data = prepare_data.read_file(test_file)
    data = test_data
    feature = Feature()
    decoder = Decoder(beam_size, feature.get_score)

    count = 0
    data_size = len(data)

    model_file = open(
        'cv-' + str(cv_epoch_num) + '-model_result/avg-model_beam-size-' + str(beam_size) + '.pkl', 'rb')
    feature.load_model(model_file)
    model_file.close()
    for line in data:
        z = decoder.beamSearch(line)
        seg_data = ' '.join(z)
        seg_data_file = 'cv-' + str(cv_epoch_num) + '-test_seg_data/avg-test-seg-data' + \
            '_beam-size-' + str(beam_size) + '.txt'
        with open(seg_data_file, 'a') as f:
            f.write(seg_data + '\n')
        count += 1
        if count % 1000 == 0:
            print("segment with avg-model, finish %.2f%%" %
                  ((count / data_size) * 100))
            logging.info("segment with avg-model, finish %.2f%%" %
                  ((count / data_size) * 100))
    f.close()
    print("segment with avg model finish")

In [8]:
# import logging

# logging.basicConfig(filename='5-fold-cv-process.log', \
#     format='%(asctime)s - %(filename)s[line:%(lineno)d] - %(levelname)s: %(message)s', level=logging.DEBUG)
# logging.info("This is a info log.")

In [9]:
import sklearn
import numpy as np
from sklearn.model_selection import KFold

import logging
import os.path
import time
import sys

from Feature import Feature
from Decoder import Decoder
import prepare_data
from evaluating_utils import evaluate


logging.basicConfig(filename='5-fold-cv-process.log', \
    format='%(asctime)s - %(levelname)s: %(message)s', level=logging.DEBUG)
logging.info("This is a info log.")

kf = KFold(n_splits=5, random_state=10, shuffle=True)

train_file = 'data/fusion_data.txt'
raw_data = prepare_data.read_file(train_file)
cv_epoch_num = 1

for train_id, test_id in kf.split(raw_data):
    logging.info("cross-valid epoch: %d", cv_epoch_num)
    print("cross-valid epoch:", cv_epoch_num)

    train_id_str = str(train_id[0:3]) + '...' + str(train_id[-3:])
    test_id_str = str(test_id[0:3]) + '...' + str(test_id[-3:])
    logging.info("TRAIN: %s, TEST: %s", train_id_str, test_id_str)
    print("TRAIN:", train_id, "TEST:", test_id)

    train_data, test_data = np.array(raw_data)[train_id], np.array(raw_data)[test_id]

    gold_test_filepath = "data/cv-" + str(cv_epoch_num) + "-filter_test.txt"
    prepare_data.prepare_data(test_data, gold_test_filepath)

    logging.info("len of raw_data: %d, train_data: %d, test_data: %d", \
        len(raw_data), len(train_data), len(test_data))
    print("len of raw_data:", len(raw_data), "train_data:", len(train_data), "test_data:", len(test_data))
    logging.info("-------------------------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------------------------")
    cv_epoch_num += 1
    # break


cross-valid epoch: 1
TRAIN: [    1     2     3 ... 38326 38327 38328] TEST: [    0     7     9 ... 38307 38310 38315]
file filtering completed
len of raw_data: 38329 train_data: 30663 test_data: 7666
-------------------------------------------------------------------------------------------------
cross-valid epoch: 2
TRAIN: [    0     1     2 ... 38326 38327 38328] TEST: [    5    12    13 ... 38320 38321 38324]
file filtering completed
len of raw_data: 38329 train_data: 30663 test_data: 7666
-------------------------------------------------------------------------------------------------
cross-valid epoch: 3
TRAIN: [    0     2     3 ... 38325 38326 38328] TEST: [    1    26    31 ... 38319 38323 38327]
file filtering completed
len of raw_data: 38329 train_data: 30663 test_data: 7666
-------------------------------------------------------------------------------------------------
cross-valid epoch: 4
TRAIN: [    0     1     2 ... 38324 38326 38327] TEST: [    3     8    10 ... 38322 3

In [10]:
kf = KFold(n_splits=5, random_state=10, shuffle=True)

train_file = 'data/fusion_data.txt'
raw_data = prepare_data.read_file(train_file)
cv_epoch_num = 1

for train_id, test_id in kf.split(raw_data):
    print("Start cross-validation.")
    logging.info("Start cross-validation.")
    logging.info("cross-valid epoch: %d", cv_epoch_num)
    print("cross-valid epoch:", cv_epoch_num)
    
    train_data, test_data = np.array(raw_data)[train_id], np.array(raw_data)[test_id]

    train_avg(iterations=3, train_data=train_data, beam_size=16, cv_epoch_num=cv_epoch_num)
    print("-------------------------------------------------------------------------------------------------")
    test_avg(iterations=3, test_data=test_data, beam_size=16, cv_epoch_num=cv_epoch_num)
    print("-------------------------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------------------------")

    cv_epoch_num += 1
    # print(len(raw_data), len(train_data), len(test_data))
    # break

Start cross-validation.
cross-valid epoch: 1
iter 0 , finish 3.26%


In [ ]:
from evaluating_utils import evaluate

cv_iter = 1
beam_size = 16
for cv_iter in range(1, 6):
    test_filepath = 'cv-' + str(cv_iter) + '-test_seg_data/avg-test-seg-data' + '_beam-size-' + str(beam_size) + '.txt'
    gold_test_filepath = "data/cv-" + str(cv_iter) + "-filter_test.txt"
    word_precision, word_recall, word_fmeasure = evaluate(test_filepath, gold_test_filepath)
    
    logging.info("cv-%d-eval:", cv_iter)
    logging.info("precision: %.3f" % word_precision)
    logging.info("recall: %.3f" % word_recall)
    logging.info("F1: %.3f" % word_fmeasure)
    logging.info("-------------------------------------------------------------")

    cv_iter += 1
